RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

c:\Users\Biju\Desktop\Python_Projects\RAG_Application\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 2 PDF files to process

Processing: Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf
  ✓ Loaded 1 pages

Processing: Alva Labs Personality Test Report - Sowmya Ayarottupura Velayudhankutty.pdf
  ✓ Loaded 6 pages

Total documents loaded: 7


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Skia/PDF m117', 'creator': 'Chromium', 'creationdate': '2024-10-08T13:06:17+00:00', 'moddate': '2024-10-08T13:06:17+00:00', 'source': '..\\data\\Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'file_type': 'pdf'}, page_content='Sowmya Ayarottupura Velayudhankutty\nSowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws accurateconclusions from it.\nSlightly above average84th - 93rd percentile.\nYour score on the logic test is 8, which is slightlyabove average.\nAlva Labs interprets logical ability bybenchmarking your score against others in theworking population. A score of slightly aboveaverage indicates a person often requires lesstime and effort to interpret abstract informationand form log

In [4]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [5]:
chunks=split_documents(all_pdf_documents)
chunks

Split 7 documents into 15 chunks

Example chunk:
Content: Sowmya Ayarottupura Velayudhankutty
Sowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws acc...
Metadata: {'producer': 'Skia/PDF m117', 'creator': 'Chromium', 'creationdate': '2024-10-08T13:06:17+00:00', 'moddate': '2024-10-08T13:06:17+00:00', 'source': '..\\data\\Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Skia/PDF m117', 'creator': 'Chromium', 'creationdate': '2024-10-08T13:06:17+00:00', 'moddate': '2024-10-08T13:06:17+00:00', 'source': '..\\data\\Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'file_type': 'pdf'}, page_content='Sowmya Ayarottupura Velayudhankutty\nSowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws accurateconclusions from it.\nSlightly above average84th - 93rd percentile.\nYour score on the logic test is 8, which is slightlyabove average.\nAlva Labs interprets logical ability bybenchmarking your score against others in theworking population. A score of slightly aboveaverage indicates a person often requires lesstime and effort to interpret abstract informationand form log

embedding And vectorStoreDB

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


VectorStore

In [8]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 45


In [9]:
chunks

[Document(metadata={'producer': 'Skia/PDF m117', 'creator': 'Chromium', 'creationdate': '2024-10-08T13:06:17+00:00', 'moddate': '2024-10-08T13:06:17+00:00', 'source': '..\\data\\Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Alva Labs Logic Test Report - Sowmya Ayarottupura Velayudhankutty.pdf', 'file_type': 'pdf'}, page_content='Sowmya Ayarottupura Velayudhankutty\nSowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws accurateconclusions from it.\nSlightly above average84th - 93rd percentile.\nYour score on the logic test is 8, which is slightlyabove average.\nAlva Labs interprets logical ability bybenchmarking your score against others in theworking population. A score of slightly aboveaverage indicates a person often requires lesstime and effort to interpret abstract informationand form log

In [10]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 15 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]

Generated embeddings with shape: (15, 384)
Adding 15 documents to vector store...
Successfully added 15 documents to vector store
Total documents in collection: 60


Retriever Pipeline From VectorStore

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [12]:
rag_retriever

In [13]:
rag_retriever.retrieve("What is the result of personality test")

Retrieving documents for query: 'What is the result of personality test'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 40.35it/s]

Generated embeddings with shape: (1, 384)
Retrieved 4 documents (after filtering)


[{'id': 'doc_132ccab6_3',
  'content': "Sowmya Ayarottupura Velayudhankutty\nSowmya Ayarottupura Velayudhankutty - Personality profile\nAlva’s personality test is constructed and built on the so-called\n'Five Factor Personality Theory', which is considered the most\nvalid framework and model for quantifying personality.\nFACTOR LOWER BAND AVERAGE UPPER BAND\nAgreeableness Detached Friendly\nConscientiousness Relaxed Diligent\nExtraversion Reserved Outgoing\nEmotional Stability Sensitive Resilient\nOpenness to Experience Conventional Innovative\nPossible strengths\n• Works hard to benefit the team and achieve\ncommon goals\n•Warm and friendly 'people person'; makes\nothers comfortable\n• Highly productive and able to work at a high\ntempo\nPossible challenges\n• Tends to have a hard time saying no and limit\ntheir workload\n• May forget about own needs while attending\nto others\n• Could come across as pushy when wanting\nthings done\nGrowth factors\nDrivers\nWhat are the key motivators

In [14]:
rag_retriever.retrieve("sowmya's logical test result")

Retrieving documents for query: 'sowmya's logical test result'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 36.02it/s]

Generated embeddings with shape: (1, 384)
Retrieved 4 documents (after filtering)


[{'id': 'doc_1d1705aa_0',
  'content': 'Sowmya Ayarottupura Velayudhankutty\nSowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws accurateconclusions from it.\nSlightly above average84th - 93rd percentile.\nYour score on the logic test is 8, which is slightlyabove average.\nAlva Labs interprets logical ability bybenchmarking your score against others in theworking population. A score of slightly aboveaverage indicates a person often requires lesstime and effort to interpret abstract informationand form logical conclusions. People with similarscores often experience they can easily acquireand apply new knowledge, provided the topic isnot complicated.Keep in mind when interpreting your results thatlogical ability is far from the only factordetermining job performance or career success.Personality, motivation, values, and experienceare also important, and strengths in one areamay ou

RAG Pipeline- VectorDB To LLM Output Generation

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
#from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

In [17]:
# class GroqLLM:
#     def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
#         """
#         Initialize Groq LLM
        
#         Args:
#             model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
#             api_key: Groq API key (or set GROQ_API_KEY environment variable)
#         """
#         self.model_name = model_name
#         self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        
#         if not self.api_key:
#             raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")
        
#         self.llm = ChatGroq(
#             groq_api_key=self.api_key,
#             model_name=self.model_name,
#             temperature=0.1,
#             max_tokens=1024
#         )
        
#         print(f"Initialized Groq LLM with model: {self.model_name}")

#     def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
#         """
#         Generate response using retrieved context
        
#         Args:
#             query: User question
#             context: Retrieved document context
#             max_length: Maximum response length
            
#         Returns:
#             Generated response string
#         """
        
#         # Create prompt template
#         prompt_template = PromptTemplate(
#             input_variables=["context", "question"],
#             template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.
#             Context:
# {context}

# Question: {question}

# Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
#         )
        
#         # Format the prompt
#         formatted_prompt = prompt_template.format(context=context, question=query)
        
#         try:
#             # Generate response
#             messages = [HumanMessage(content=formatted_prompt)]
#             response = self.llm.invoke(messages)
#             return response.content
            
#         except Exception as e:
#             return f"Error generating response: {str(e)}"
        
#     def generate_response_simple(self, query: str, context: str) -> str:
#         """
#         Simple response generation without complex prompting
        
#         Args:
#             query: User question
#             context: Retrieved context
            
#         Returns:
#             Generated response
#         """
#         simple_prompt = f"""Based on this context: {context}

# Question: {query}

# # Answer:"""
         
#         try:
#             messages = [HumanMessage(content=simple_prompt)]
#             response = self.llm.invoke(messages)
#             return response.content
#         except Exception as e:
#             return f"Error: {str(e)}"




class OpenAILLM:
    def __init__(self, model_name: str = "gpt-4o-mini", api_key: str = None):
        """
        Initialize OpenAI LLM
        
        Args:
            model_name: OpenAI model name (gpt-4.1, gpt-4o-mini, gpt-4.1-mini, etc.)
            api_key: OpenAI API key (or set OPENAI_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        
        if not self.api_key:
            raise ValueError("OpenAI API key is required. Set OPENAI_API_KEY or pass api_key parameter.")
        
        self.llm = ChatOpenAI(
            model=self.model_name,
            api_key=self.api_key,
            temperature=0.1,
            max_tokens=1024
        )
        
        print(f"Initialized OpenAI LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context
        """
        
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context does not contain enough information, say so."""
        )
        
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without prompt template
        """
        
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""
        
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"


In [18]:
# Initialize OpenAI LLM (you'll need to set OPENAI_API_KEY environment variable)
try:
    openai_llm = OpenAILLM(api_key=os.getenv("OPENAI_API_KEY"))
    print("OpenAI LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your OPENAI_API_KEY environment variable to use the LLM.")
    openai_llm = None    

Initialized OpenAI LLM with model: gpt-4o-mini
OpenAI LLM initialized successfully!


In [19]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("sowmya's logical test result")

Retrieving documents for query: 'sowmya's logical test result'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.12it/s]

Generated embeddings with shape: (1, 384)
Retrieved 4 documents (after filtering)


[{'id': 'doc_1d1705aa_0',
  'content': 'Sowmya Ayarottupura Velayudhankutty\nSowmya AyarottupuraVelayudhankutty - Logical abilityAlva’s logic test assesses logical ability, i.e., how efficiently oneprocesses complex information and draws accurateconclusions from it.\nSlightly above average84th - 93rd percentile.\nYour score on the logic test is 8, which is slightlyabove average.\nAlva Labs interprets logical ability bybenchmarking your score against others in theworking population. A score of slightly aboveaverage indicates a person often requires lesstime and effort to interpret abstract informationand form logical conclusions. People with similarscores often experience they can easily acquireand apply new knowledge, provided the topic isnot complicated.Keep in mind when interpreting your results thatlogical ability is far from the only factordetermining job performance or career success.Personality, motivation, values, and experienceare also important, and strengths in one areamay ou

Integration Vectordb Context pipeline With LLM output

In [20]:
### Simple RAG pipeline with OpenAI LLM
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

### Initialize the OpenAI LLM (set your OPENAI_API_KEY in environment)
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-4o-mini",     # recommended model
    temperature=0.1,
    max_tokens=1024
)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retrieve the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""

    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answer using OpenAI LLM
    prompt = f"""Use the following context to answer the question concisely.
Context:
{context}

Question: {query}

Answer:"""
    
    response = llm.invoke([prompt])
    return response.content


In [21]:
answer=rag_simple("What is her personality?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is her personality?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 43.32it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Sowmya Ayarottupura Velayudhankutty's personality can be characterized by the following traits based on the Five Factor Personality Theory:

- **Agreeableness**: Friendly and warm, making others comfortable.
- **Conscientiousness**: Diligent and hardworking, focused on team goals.
- **Extraversion**: Outgoing and sociable.
- **Emotional Stability**: Resilient, though may be sensitive at times.
- **Openness to Experience**: Innovative and open-minded.

Possible strengths include her productivity and ability to work well in a team, while challenges may involve difficulty in setting boundaries and prioritizing her own needs.


Enhanced RAG Pipeline Features

In [22]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is sowmyas personality", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is sowmyas personality'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [26]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("How to measure personality?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'How to measure personality?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 19.49it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
Sowmya Ayarottupura Velayudhankutty
Sowmya Ayarottupura Velayudhankutty - Personality profile
Alva’s personality test is constructed and built on the so-called
'Five Facto

r Personality Theory', which is considered the most
valid framework and model for quantifying personality.
FACTOR LOWER BAND AVERAGE UPPER BAND
Agreeableness Detached Friendly
Conscientiousness Relaxed Diligent
Extraversion Reserved Outgoing
Emotional Stability Sensitive Resilient
Openness to Experience Conventional Innovative
Possible strengths
• Works hard to benefit the team and achieve
common goals
•Warm and friendly 'people person'; makes
others comfortable
• Highly productive and able to work at a high
tempo
Possible challenges
• Tends to have a hard time saying no and limit
their workload
• May forget about own needs while attending
to others
• Could come across as pushy when wanting
things done
Growth factors
Drivers
What are the key motivators for this person?
Culture preferences

Sowmya Ayarottupura Velayudhankutty
Sowmya Ayarottupura Velayudhankutty - Personality profile
Alva’s personality test is constructed and built on the so-called
'Five Factor Personality Theory', which